In [1]:
import os
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

In [2]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
DATA_INTERIM_DIR = os.path.join(data_dir, 'interim')
data_raw_dir

'/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW/data/raw'

In [3]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [ ]:
spark = init_spark()
data_weather = spark.read.csv(os.path.join(data_raw_dir, 'en_climate_daily_QC_702S006_2015_P1D.csv'), header=True)
 
data_weather.schema.names

In [ ]:
with open('schema.txt', 'w') as f:
    for col in data_weather.schema.names:
        print(col, file=f)

In [4]:
# data_towing = spark.read.csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=True)
# data_towing = data_towing.withColumn('DATE_ORIGINE', F.concat(data_towing.DATE_ORIGINE.substr(0,10)))
spark = init_spark()
data_towing = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'towing.data'))
data_weather = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather.data'))

In [5]:
data_towing.head(2)

[Row(DATE_ORIGINE='2019-01-10', LONGITUDE_ORIGINE='-73.60370215594651', LATITUDE_ORIGINE='45.5590991399067', Distance_km=0.0484305337462778, MOTIF_REMORQUAGE="Constat d'infraction"),
 Row(DATE_ORIGINE='2019-01-10', LONGITUDE_ORIGINE='-73.599733540326', LATITUDE_ORIGINE='45.453884709812606', Distance_km=0.07233637828159714, MOTIF_REMORQUAGE="Constat d'infraction")]

In [6]:
data_weather.head(2)

[Row(Longitude='-73.74', Latitude='45.47', Station_Name='MONTREAL/PIERRE ELLIOTT TRUDEAU INTL', Climate_ID='702S006', Date_Time='2015-01-01', Year='2015', Month='01', Day='01', Data_Quality=None, Max_Temp='-2.9', Max_Temp_Flag=None, Min_Temp='-7.4', Min_Temp_Flag=None, Mean_Temp='-5.2', Mean_Temp_Flag=None, Heat_Deg_Days='23.2', Heat_Deg_Days_Flag=None, Cool_Deg_Days='0.0', Cool_Deg_Days_Flag=None, Total_Rain=None, Total_Rain_Flag='M', Total_Snow=None, Total_Snow_Flag='M', Total_Precip='0.3', Total_Precip_Flag=None, Snow_on_Grnd=None, Snow_on_Grnd_Flag=None, Dir_of_Max_Gust='21', Dir_of_Max_Gust_Flag=None, Spd_of_Max_Gust='54', Spd_of_Max_Gust_Flag=None),
 Row(Longitude='-73.74', Latitude='45.47', Station_Name='MONTREAL/PIERRE ELLIOTT TRUDEAU INTL', Climate_ID='702S006', Date_Time='2015-01-02', Year='2015', Month='01', Day='02', Data_Quality=None, Max_Temp='-3.3', Max_Temp_Flag=None, Min_Temp='-16.0', Min_Temp_Flag=None, Mean_Temp='-9.7', Mean_Temp_Flag=None, Heat_Deg_Days='27.7', Heat

In [ ]:
joined_data = data_towing.join(data_weather, data_towing.DATE_ORIGINE==data_weather.DateTime)
joined_data.head(1)

In [ ]:
df_final = joined_data.select('DATE_ORIGINE', 'LONGITUDE_ORIGINE', 'LATITUDE_ORIGINE', 'Mean Temp (°C)', 'Total Precip (mm)', 'Snow on Grnd (cm)')